In [ ]:
%matplotlib inline


# Harmonic-percussive source separation

This notebook illustrates how to separate an audio signal into
its harmonic and percussive components.

We'll compare the original median-filtering based approach of
`Fitzgerald, 2010 <http://arrow.dit.ie/cgi/viewcontent.cgi?article=1078&context=argcon>`_
and its margin-based extension due to `Dreidger, Mueller and Disch, 2014
<http://www.terasoft.com.tw/conf/ismir2014/proceedings/T110_127_Paper.pdf>`_.


In [4]:
# ran into this issue
# https://github.com/librosa/librosa/issues/219
# if using mac/linux I had to install ffmpeg to parse the audio. If using windows you might need to do something else. Librosa uses a 3rd party app to load audio files, and it doesn't tell you how to fix errors if there are any with loading 
# the audio files. 

import numpy as np
import matplotlib.pyplot as plt

import librosa
import librosa.display
import sklearn

import numpy as np
from collections import Counter, defaultdict
import time
import math
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

import xml.dom.minidom
import os
from os.path import exists
from collections import defaultdict

## Below is the example code on how to utilize Librosa with harmonic-percussive source seperation.

Load an example clip with harmonics and percussives



In [ ]:
y, sr = librosa.load('chocomint.flac', duration=5, offset=10)

Compute the short-time Fourier transform of y



In [ ]:
D = librosa.stft(y)

Decompose D into harmonic and percussive components

$D = D_\text{harmonic} + D_\text{percussive}$



In [ ]:
D_harmonic, D_percussive = librosa.decompose.hpss(D)

We can plot the two components along with the original spectrogram



In [ ]:
# Pre-compute a global reference power from the input spectrum
rp = np.max(np.abs(D))

fig, ax = plt.subplots(nrows=3, sharex=True, sharey=True)

img = librosa.display.specshow(librosa.amplitude_to_db(np.abs(D), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[0])
ax[0].set(title='Full spectrogram')
ax[0].label_outer()

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_harmonic), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[1])
ax[1].set(title='Harmonic spectrogram')
ax[1].label_outer()

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_percussive), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[2])
ax[2].set(title='Percussive spectrogram')
fig.colorbar(img, ax=ax)

The default HPSS above assigns energy to each time-frequency bin according to
whether a horizontal (harmonic) or vertical (percussive) filter responds higher
at that position.

This assumes that all energy belongs to either a harmonic or percussive source,
but does not handle "noise" well.  Noise energy ends up getting spread between
D_harmonic and D_percussive.

If we instead require that the horizontal filter responds more than the vertical
filter *by at least some margin*, and vice versa, then noise can be removed
from both components.

Note: the default (above) corresponds to margin=1



In [ ]:
# Let's compute separations for a few different margins and compare the results below
D_harmonic2, D_percussive2 = librosa.decompose.hpss(D, margin=2)
D_harmonic4, D_percussive4 = librosa.decompose.hpss(D, margin=4)
D_harmonic8, D_percussive8 = librosa.decompose.hpss(D, margin=8)
D_harmonic16, D_percussive16 = librosa.decompose.hpss(D, margin=16)

In the plots below, note that vibrato has been suppressed from the harmonic
components, and vocals have been suppressed in the percussive components.



In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=2, sharex=True, sharey=True, figsize=(10, 10))
librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_harmonic), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[0, 0])
ax[0, 0].set(title='Harmonic')

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_percussive), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[0, 1])
ax[0, 1].set(title='Percussive')
print(D_percussive)

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_harmonic2), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[1, 0])

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_percussive2), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[1, 1])

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_harmonic4), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[2, 0])

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_percussive4), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[2, 1])

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_harmonic8), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[3, 0])

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_percussive8), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[3, 1])

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_harmonic16), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[4, 0])

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_percussive16), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[4, 1])

for i in range(5):
    ax[i, 0].set(ylabel='margin={:d}'.format(2**i))
    ax[i, 0].label_outer()
    ax[i, 1].label_outer()

## Below is where code starts for 0x40 hues to parse respacks and audio.

In [5]:
# parse for the song names and folders
respack_folders = []
for directory, sub_directories, files in os.walk('./Respacks/'):
    for folder in sub_directories:
        if folder != 'Songs' and folder != 'characters' and folder != 'Images':
            respack_folders.append(folder)
print(respack_folders)

['IntegralStoryMix', 'Xmas', 'HuesMixB', 'Royalty', 'Thistle', 'WeekendInsideMix', 'AprilHues', 'ShuttersPack', "Tylup's pack", 'HuesMixA', 'PackShit', '420', '.ipynb_checkpoints', 'DefaultsHQ', '.ipynb_checkpoints', 'Yukata', '.ipynb_checkpoints', 'snoop', '.ipynb_checkpoints', '.ipynb_checkpoints', 'Animations', 'Bob']


In [6]:
# Parse and get the rhythm and build up rhythm maps.
rhythm_map = defaultdict(str)
build_up_rhythm_map = defaultdict(str)

for folder in respack_folders:
    path_name = './Respacks/' + folder + '/songs.xml'
    print(path_name)
    if exists(path_name):
        song_xml = xml.dom.minidom.parse(path_name)
        songs = song_xml.getElementsByTagName('song')
        for song in songs:
            # parsing rhythms
            curr_song_name = song.getAttribute('name')
            if len(song.getElementsByTagName('rhythm')) > 0:
                rhythm_map[curr_song_name] = song.getElementsByTagName('rhythm')[0].firstChild.nodeValue
            if len(song.getElementsByTagName('buildup')) > 0:
                build_up_name = song.getElementsByTagName('buildup')[0].firstChild.nodeValue
                if len(song.getElementsByTagName('buildupRhythm')) > 0:
                    build_up_rhythm_map[build_up_name] = song.getElementsByTagName('buildupRhythm')[0].firstChild.nodeValue
            
    else:
        print('bad path name ' + str(path_name))

print(rhythm_map.items())
print(build_up_rhythm_map.items())

./Respacks/IntegralStoryMix/songs.xml
./Respacks/Xmas/songs.xml
./Respacks/HuesMixB/songs.xml
./Respacks/Royalty/songs.xml
./Respacks/Thistle/songs.xml
./Respacks/WeekendInsideMix/songs.xml
./Respacks/AprilHues/songs.xml
./Respacks/ShuttersPack/songs.xml
./Respacks/Tylup's pack/songs.xml
./Respacks/HuesMixA/songs.xml
./Respacks/PackShit/songs.xml
./Respacks/420/songs.xml
./Respacks/.ipynb_checkpoints/songs.xml
bad path name ./Respacks/.ipynb_checkpoints/songs.xml
./Respacks/DefaultsHQ/songs.xml
./Respacks/.ipynb_checkpoints/songs.xml
bad path name ./Respacks/.ipynb_checkpoints/songs.xml
./Respacks/Yukata/songs.xml
bad path name ./Respacks/Yukata/songs.xml
./Respacks/.ipynb_checkpoints/songs.xml
bad path name ./Respacks/.ipynb_checkpoints/songs.xml
./Respacks/snoop/songs.xml
bad path name ./Respacks/snoop/songs.xml
./Respacks/.ipynb_checkpoints/songs.xml
bad path name ./Respacks/.ipynb_checkpoints/songs.xml
./Respacks/.ipynb_checkpoints/songs.xml
bad path name ./Respacks/.ipynb_checkpoi

In [7]:
# Parse and extract features from audio and build dictionary with song_name -> audio features
        
def parseSong(y, sr):
    T = 30.0    # seconds
    t = np.linspace(0, T, int(T*sr), endpoint=False) # time variable
    x = 0.5*np.sin(2*np.pi*220*t)# pure sine wave at 220 Hz
    D = librosa.stft(y)
    ret_dict = dict()
    ret_dict['D_harmonic4'], ret_dict['D_percussive4'] = librosa.decompose.hpss(D, margin=4)
    ret_dict['D_harmonic16'], ret_dict['D_percussive16'] = librosa.decompose.hpss(D, margin=16)
    ret_dict['spectral_centroids'] = librosa.feature.spectral_centroid(x, sr=sr)[0]
    ret_dict['spectral_rolloff'] = librosa.feature.spectral_rolloff(x+0.01, sr=sr)[0]
    
    
    #spectral_bandwidth_2 = librosa.feature.spectral_bandwidth(x+0.01, sr=sr)[0]
    #spectral_bandwidth_3 = librosa.feature.spectral_bandwidth(x+0.01, sr=sr, p=3)[0]
    #spectral_bandwidth_4 = librosa.feature.spectral_bandwidth(x+0.01, sr=sr, p=4)[0]
    #ret_dict['spectral_bandwidth_2'] = spectral_bandwidth_2
    #ret_dict['spectral_bandwidth_3'] = spectral_bandwidth_3
    #ret_dict['spectral_bandwidth_4'] = spectral_bandwidth_4
    return ret_dict

# dict(audio_name -> dict(features -> values))
audio_rhythm_feature_map = defaultdict(dict)
audio_build_up_rhythm_feature_map = defaultdict(dict)

for folder in respack_folders:
    path_name = './Respacks/' + folder + '/songs.xml'
    print(path_name)
    if exists(path_name):
        song_xml = xml.dom.minidom.parse(path_name)
        songs = song_xml.getElementsByTagName('song')
        for song in songs:
            # parsing rhythms
            curr_song_name = song.getAttribute('name')
            song_path_name = None
            is_rhythm = False
            is_build_up = False
            if len(song.getElementsByTagName('rhythm')) > 0:
                is_rhythm = True
                song_path_name = path_name = './Respacks/' + folder +'/Songs/' + curr_song_name + '.mp3'
            
            if len(song.getElementsByTagName('buildup')) > 0:
                is_build_up - True
                build_up_name = song.getElementsByTagName('buildup')[0].firstChild.nodeValue
                song_path_name = path_name = './Respacks/' + folder +'/Songs/' + build_up_name + '.mp3'
                    
            # load the song
            if exists(song_path_name):
                print('parsing ' + str(song_path_name))
                try:
                    y, sr = librosa.load(song_path_name, duration=5, offset=10)
                    if is_rhythm:
                        audio_rhythm_feature_map[curr_song_name] = parseSong(y,sr)
                    elif is_build_up:
                        audio_build_up_rhythm_feature_map[build_up_name] = parseSong(y,sr)
                except Exception as e:
                    print('exception occurred')
                    print(e)
            else:
                print('file name doesn\'t exist ' + rhythm_song_path_name)
    else:
        print('bad path name ' + str(path_name))
        

./Respacks/IntegralStoryMix/songs.xml
parsing ./Respacks/IntegralStoryMix/Songs/Netsky - Puppy (intro).mp3


/Users/frankhu/opt/anaconda3/envs/ai_env/lib/python3.7/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


parsing ./Respacks/IntegralStoryMix/Songs/Blake McGrath- Motion Picture (Pegboard Nerds Remix)_Build.mp3


/Users/frankhu/opt/anaconda3/envs/ai_env/lib/python3.7/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


parsing ./Respacks/IntegralStoryMix/Songs/lrad.mp3


...

/Users/frankhu/opt/anaconda3/envs/ai_env/lib/python3.7/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


parsing ./Respacks/DefaultsHQ/Songs/build_Weapon.mp3
exception occurred
Input signal length=0 is too small to resample from 44100->22050
parsing ./Respacks/DefaultsHQ/Songs/build_LoveOnHaightStreet.mp3


/Users/frankhu/opt/anaconda3/envs/ai_env/lib/python3.7/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/Users/frankhu/opt/anaconda3/envs/ai_env/lib/python3.7/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


exception occurred
Input signal length=0 is too small to resample from 44100->22050
parsing ./Respacks/DefaultsHQ/Songs/build_TheClockmaker.mp3
exception occurred
Input signal length=0 is too small to resample from 44100->22050
./Respacks/.ipynb_checkpoints/songs.xml
bad path name ./Respacks/.ipynb_checkpoints/songs.xml
./Respacks/Yukata/songs.xml
bad path name ./Respacks/Yukata/songs.xml
./Respacks/.ipynb_checkpoints/songs.xml
bad path name ./Respacks/.ipynb_checkpoints/songs.xml
./Respacks/snoop/songs.xml
bad path name ./Respacks/snoop/songs.xml
./Respacks/.ipynb_checkpoints/songs.xml
bad path name ./Respacks/.ipynb_checkpoints/songs.xml
./Respacks/.ipynb_checkpoints/songs.xml
bad path name ./Respacks/.ipynb_checkpoints/songs.xml
./Respacks/Animations/songs.xml
bad path name ./Respacks/Animations/songs.xml
./Respacks/Bob/songs.xml
bad path name ./Respacks/Bob/songs.xml


/Users/frankhu/opt/anaconda3/envs/ai_env/lib/python3.7/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [ ]:
index = 0
total_files_to_print = 10
for folder in respack_folders:
    if index == total_files_to_print:
        break
    path_name = './Respacks/' + folder + '/songs.xml'
    print(path_name)
    if exists(path_name):
        song_xml = xml.dom.minidom.parse(path_name)
        songs = song_xml.getElementsByTagName('song')
        for song in songs:
            index += 1
            if index == total_files_to_print:
                break
            # parsing rhythms
            curr_song_name = song.getAttribute('name')
            song_path_name = None
            is_rhythm = False
            is_build_up = False
            if len(song.getElementsByTagName('rhythm')) > 0:
                is_rhythm = True
                song_path_name = path_name = './Respacks/' + folder +'/Songs/' + curr_song_name + '.mp3'
            
            if len(song.getElementsByTagName('buildup')) > 0:
                is_build_up - True
                build_up_name = song.getElementsByTagName('buildup')[0].firstChild.nodeValue
                song_path_name = path_name = './Respacks/' + folder +'/Songs/' + build_up_name + '.mp3'
                    
            # load the song
            if exists(song_path_name):
                print('outputting features for ' + str(song_path_name))
                if is_rhythm:
                    for key in audio_rhythm_feature_map[curr_song_name].keys():
                        print(key)
                        print(len(audio_rhythm_feature_map[curr_song_name][key]))
                elif is_build_up:
                    print(audio_build_up_rhythm_feature_map[build_up_name])
            else:
                print('file name doesn\'t exist ' + rhythm_song_path_name)
    else:
        print('bad path name ' + str(path_name))
        

In [8]:
# evaluate random forest algorithm for classification

# features = audio features
# labels = rhythm/buildup
features = []
labels = []
for song_name in rhythm_map.keys():
    if song_name in audio_rhythm_feature_map:
        labels.append(rhythm_map[song_name])
        features.append([])
        for feature in audio_rhythm_feature_map[song_name]:
            if feature in audio_rhythm_feature_map[song_name]:
                for feature_index in range(len(audio_rhythm_feature_map[song_name][feature])):
                    x = audio_rhythm_feature_map[song_name][feature][feature_index]
                    complex_to_real = x.real + x.imag
                    features[-1].append(complex_to_real)
print(len(labels))
print(len(features))
# print(total/130)
# audio_rhythm_features = audio_rhythm_feature_map.values()
# audio_rhythm_labels = rhythm_map.values()

72
72


In [ ]:
post_processed_labels = []
post_processed_features = []
print(len(features))
print(len(labels))
for index in range(len(labels)):
    for beat_index in range(len(labels[index])):
        post_processed_labels.append(labels[index][beat_index])
        post_processed_features.append([])
        complex_to_real = 0 
        post_processed_features[-1].append(complex_to_real)
        for feature_index in range(len(features[index])):
            x = features[index][feature_index]
            complex_to_real = x.real + x.imag
            post_processed_features[-1].append(complex_to_real)
            # print(complex_to_real)
            
    #print(len(post_processed_labels[index]))
    #print(len(post_processed_features[index]))
print(len(post_processed_labels))
print(len(post_processed_features[0]))

72
72


In [ ]:
# define the model
model = RandomForestClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, features, labels, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
for index in range(len(post_processed_labels)):
    print(len(post_processed_labels[index]))
    print(len(post_processed_features[index]))

In [ ]:
model.fit(np.array(features), np.array(labels))

In [ ]:

# dict(audio_name -> dict(features -> values))
path_name = 'chocomint.flac'
print(path_name)
if exists(path_name):
    print('parsing ' + str(path_name))
    try:
        y, sr = librosa.load(path_name, duration=5, offset=10)
        single_song = parseSong(y,sr)
    except Exception as e:
        print('exception occurred')
        print(e)
else:
    print('bad path name ' + str(path_name))

In [ ]:
single_feature_input = []
for feature in single_song:
    for feature_index in range(len(single_song[feature])):
        x = single_song[feature][feature_index]
        complex_to_real = x.real + x.imag
        single_feature_input.append(complex_to_real)
'''
for index in range(len(single_feature_input)):
    model.predict(np.array([single_feature_input[index][:2]]))
'''
model.predict(np.array([single_feature_input]))


In [ ]:
forest = ChallengeClassifier()
print(len(features))
print(len(labels))
forest.fit(features, labels)
output = forest.classify(features)